In [2]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 데이터 로드
train_data = pd.read_csv('train.csv')

# 누락된 AlogP 값이 있는 행 제거
# train = train_data.dropna(subset=['AlogP'])
median_value = train_data['AlogP'].median()
train_data['AlogP'].fillna(median_value, inplace=True)
train_data_cleaned = train_data
# SMILES 문자열에서 fingerprint를 생성하는 함수
def generate_fingerprint(smiles_string):
    mol = Chem.MolFromSmiles(smiles_string)
    if mol is not None:
        fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        return list(fingerprint)
    else:
        return [0]*2048

# 모든 SMILES 문자열에 대해 fingerprint를 생성
fingerprint_data = train_data_cleaned['SMILES'].apply(generate_fingerprint)
fingerprint_df = pd.DataFrame(fingerprint_data.tolist(), columns=[f'bit_{i}' for i in range(2048)])

# 인덱스 재설정
train_data_cleaned = train_data_cleaned.reset_index(drop=True)

# 특성 및 대상 변수 준비
features = pd.concat([train_data_cleaned.drop(columns=['id', 'SMILES', 'MLM', 'HLM']), fingerprint_df], axis=1)
target_mlm = train_data_cleaned['MLM']

# 데이터 분할
X_train, X_test, y_train_mlm, y_test_mlm = train_test_split(features, target_mlm, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델로 MLM 훈련
rf_mlm = RandomForestRegressor(n_estimators=100, random_state=42)
rf_mlm.fit(X_train, y_train_mlm)

# 테스트 세트에서 예측
y_pred_mlm = rf_mlm.predict(X_test)

# MLM의 RMSE 계산
rmse_mlm = mean_squared_error(y_test_mlm, y_pred_mlm, squared=False)
print(f"MLM의 RMSE: {rmse_mlm}")


MLM의 RMSE: 32.940506263717005


In [3]:
# HLM에 대한 타겟 변수 준비
target_hlm = train_data_cleaned['HLM']

# 데이터 분할 (HLM)
X_train_hlm, X_test_hlm, y_train_hlm, y_test_hlm = train_test_split(features, target_hlm, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델로 HLM 훈련
rf_hlm = RandomForestRegressor(n_estimators=100, random_state=42)
rf_hlm.fit(X_train_hlm, y_train_hlm)

# 테스트 세트에서 HLM 예측
y_pred_hlm = rf_hlm.predict(X_test_hlm)

# HLM의 RMSE 계산
rmse_hlm = mean_squared_error(y_test_hlm, y_pred_hlm, squared=False)
print(f"HLM의 RMSE: {rmse_hlm}")


HLM의 RMSE: 32.3970004454479


In [6]:
# test.csv 로드
test_data = pd.read_csv('test.csv')

median_value = test_data['AlogP'].median()
test_data['AlogP'].fillna(median_value, inplace=True)
# fingerprint 생성
fingerprint_data_test = test_data['SMILES'].apply(generate_fingerprint)
fingerprint_df_test = pd.DataFrame(fingerprint_data_test.tolist(), columns=[f'bit_{i}' for i in range(2048)])

# 테스트 데이터의 특성 준비
id_col = test_data['id']
test_features = pd.concat([test_data.drop(columns=['id', 'SMILES']), fingerprint_df_test], axis=1)

# MLM 및 HLM 예측
mlm_predictions = rf_mlm.predict(test_features)
hlm_predictions = rf_hlm.predict(test_features)

# 예측 값을 DataFrame으로 변환
predictions_df = pd.DataFrame({
    'id': id_col,
    'MLM': mlm_predictions,
    'HLM': hlm_predictions
})

predictions_df

,id,MLM,HLM
0,TEST_000,21.981080,39.96060
1,TEST_001,72.621300,83.38973
2,TEST_002,30.035430,41.36856
3,TEST_003,53.920577,50.30648
4,TEST_004,59.906300,75.08114
...,...,...,...
478,TEST_478,24.051193,22.38892
479,TEST_479,77.234600,94.42205
480,TEST_480,27.145823,67.73929
481,TEST_481,61.800270,78.46057


In [7]:
# 예측 값을 CSV 파일로 저장
predictions_df.to_csv('RF_AllChem_predictions.csv', index=False)